## Cleaning and Normalization

In [158]:
import pandas as pd 
import numpy as np

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
# for returning lat long coordinates from addresses
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

# since we've pickled the dataframe after attempting to geocode…
# …we'll read the pickle and not the csv
#df = pd.read_csv('jersey_city_residential.csv')

df = pd.read_pickle('JC_residential.pkl')
errors_df = pd.read_csv('errors.csv')

# number of times owner appears in second column
def propertiesOwned(owner=str):
    return len(df[df.ownersName == owner])

# number of times property appears in first column
def propertyAppearances(address=str): 
    return len(df[df.propertyLocation == address])

df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
df['propertyFullAddress'] = df.propertyLocation + ' Jersey City, NJ'


In [159]:
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-REAR', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('(REAR)', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-FRONT', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAMPTON CT.', 'HAMPTON COURT')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\(\)', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VARCK', 'VARICK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('OLD BERGEN', 'OLD BERGEN AVE')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THO', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AVE', 'AVE')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TER', 'TR')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3633A', '3633')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3144A', '3144')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SHEARWATR', 'SHEARWATER')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PALUSL', "PAUL'S")
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COURT TR', "CT")
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EASTVIEW', "E VIEW")
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DR.', 'MARTIN LUTHER KING DR.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 1ST FL.', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' SO.', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' N.', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ODGEN', 'OGDEN')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATRSON', 'PATERSON')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 61 COLES', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAN HOUTEN ST.', 'VAN HOUTEN AVE.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}(.?\d{1}(A)?)?A?\b', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\s#\d', '')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON ST.', 'MCPHERSON PL.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON', 'MCPHERSON')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIR ROAD', 'FIR ST.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('UNION AVE.', 'UNION ST.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('OLD BERGEN', 'OLD BERGEN AVE.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE. AVE.', 'AVE.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('JACKSON AVE.', 'JACKSON ST.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('162 OLD BERGEN AVE. AVE.', '162 OLD BERGEN AVE.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLEN', 'GLENN')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLENNN', 'GLENN')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('100 SHEARWATER CT.', '100 EAST SHEARWATER CT.')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('1 GLENNN LANE', '1 GLENN LANE')

errors_df.propertyFullAddress = errors_df.propertyLocation + ' Jersey City, NJ'

/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1625/305007133.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace('(REAR)', '')
/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1625/305007133.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAMPTON CT.', 'HAMPTON COURT')
/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1625/305007133.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\(\)', '')
/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1625/305007133.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = err

In [160]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

CPU times: user 378 ms, sys: 54.6 ms, total: 433 ms
Wall time: 1min 58s


In [161]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [162]:
errors_df.iloc[geocode_errors].propertyLocation.unique()

array(['78A RNE ST.', '280 GREGORY PARK PLAZA', '270 GREGORY PARK PLAZA',
       '107 SHEARWATER CT.'], dtype=object)

In [163]:
errors_df.iloc[geocode_errors]

,Unnamed: 0,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
4,2529,78A RNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A RNE ST. Jersey City, NJ",1,1,None
113,19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
114,19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
186,38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [164]:
# we've already added these columns, so we'll comment them out
#df['propertiesOwned'] = [propertiesOwned(owner) for owner in df.ownersName]
#df['units'] = [propertyAppearances(address) for address in df.propertyLocation]
#df['gCode'] = df.propertyFullAddress.apply(geolocator.geocode)

### Before we save the smaller errors data frame, we need to append the cleaned errors to the original df

In [165]:
df = df.append(errors_df.dropna(axis=0, subset=['gCode']))

In [167]:
df.to_pickle('JC_residential.pkl')

In [168]:
df.to_csv('JS_residential.csv', index=None)

In [169]:
# since we got an error saying we're writing on the dataframe…
# we'll creata a separate df of the errors and merge with the original df later

# errors_df = df.iloc[geocode_errors].copy()

# this is how we originally got errors_df, but we've corrected many errors since
# we'll instead save a copy of a copy, but only of those addresses we've yet to geocode
errors_df =  errorsdf.iloc[geocode_errors].copy()

#errors_df.to_csv('errors.csv')

In [170]:
errors_df

,Unnamed: 0,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
4,2529,78A RNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A RNE ST. Jersey City, NJ",1,1,None
113,19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
114,19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
186,38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [157]:
errors_df.to_csv('errors.csv')

,Unnamed: 0,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
4,2529,78A RNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A RNE ST. Jersey City, NJ",1,1,None
113,19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
114,19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
186,38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [3]:
df.dropna(axis=0, subset=['gCode'], inplace=True)

In [32]:
df.gCode

0        (677, Liberty Avenue, Jersey City, Hudson Coun...
1        (675, Liberty Avenue, Jersey City, Hudson Coun...
2        (673, Liberty Avenue, Jersey City, Hudson Coun...
3        (671, Liberty Avenue, Jersey City, Hudson Coun...
4        (669, Liberty Avenue, Jersey City, Hudson Coun...
                               ...                        
41799    (98, Garfield Avenue, Communipaw, Jersey City,...
41800    (100, Garfield Avenue, Jersey City, Hudson Cou...
41801    (102, Garfield Avenue, Jersey City, Hudson Cou...
41802    (104, Garfield Avenue, Jersey City, Hudson Cou...
41803    (106, Garfield Avenue, Jersey City, Hudson Cou...
Name: gCode, Length: 41804, dtype: object

In [9]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [14]:
# number of addresses address which did not geocode
geocode_errors = [i for i,e in enumerate(df.gCode) if e == None]

In [15]:
# plug them into iloc to return a dataframe of errors
df.iloc[geocode_errors]['propertyFullAddress'].head(50)

12                653 LIBERTY ST. Jersey City, NJ
112     3709 JOHN F KENNEDY BLVD. Jersey City, NJ
113     3707 JOHN F KENNEDY BLVD. Jersey City, NJ
114     3705 JOHN F KENNEDY BLVD. Jersey City, NJ
115     3703 JOHN F KENNEDY BLVD. Jersey City, NJ
175     3697 JOHN F KENNEDY BLVD. Jersey City, NJ
177     3691 JOHN F KENNEDY BLVD. Jersey City, NJ
178     3687 JOHN F KENNEDY BLVD. Jersey City, NJ
179     3685 JOHN F KENNEDY BLVD. Jersey City, NJ
180     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
181     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
182     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
183     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
184     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
185     3683 JOHN F KENNEDY BLVD. Jersey City, NJ
263     3675 JOHN F KENNEDY BLVD. Jersey City, NJ
264     3673 JOHN F KENNEDY BLVD. Jersey City, NJ
265     3671 JOHN F KENNEDY BLVD. Jersey City, NJ
266     3669 JOHN F KENNEDY BLVD. Jersey City, NJ
268     3663 JOHN F KENNEDY BLVD. Jersey City, NJ


In [52]:
# the reason why this didn't gecode properly is because there is no
# Liberty Street in Jersey City
df.propertyFullAddress[12]

'653 LIBERTY ST. Jersey City, NJ'

In [56]:
geolocator.geocode('653 LIBERTY AVE, Jersey City, NJ')

Location(653, Liberty Avenue, Jersey City, Hudson County, New Jersey, 07307, United States, (40.75896345454545, -74.05096481818181, 0.0))

In [63]:
# There is no "Kennedy BLVD" but a John F Kennedy BLVD
df.propertyFullAddress[112:115]

112    3709 KENNEDY BLVD. Jersey City, NJ
113    3707 KENNEDY BLVD. Jersey City, NJ
114    3705 KENNEDY BLVD. Jersey City, NJ
Name: propertyFullAddress, dtype: object

In [71]:
#the hyphenated address threw off the geolocator
geolocator.geocode('274 COLUMBIA AVE Jersey City, NJ')

Location(274, Columbia Avenue, Jersey City, Hudson County, New Jersey, 07307, United States, (40.75704381395349, -74.05096265116279, 0.0))

In [8]:
# the geolocator needs to the full street name
geolocator.geocode('3709 JOHN F KENNEDY BLVD. Jersey City, NJ')

Location(John F. Kennedy Boulevard, Greenville, Jersey City, Hudson County, New Jersey, 07305, United States, (40.70124, -74.093112, 0.0))

In [ ]:
df.propertyLocation = df.propertyLocation.str.replace('KENNEDY', 'JOHN F KENNEDY')
df.propertyLocation = df.propertyLocation.str.replace('LIBERTY ST.', 'LIBERTY AVE.')

/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_20899/739789649.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.propertyLocation = df.propertyLocation.str.replace('LIBERTY ST.', 'LIBERTY AVE.')


In [20]:
df.iloc[geocode_errors]['gCode'] = df.iloc[geocode_errors]['propertyFullAddress'].apply(geolocator.geocode)

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [72]:
# pickling the dataframe to avoid waiting hours for the geocodes again
# df.to_pickle('JC_residential.pkl')

In [29]:
geocode_errors = [i for i,e in enumerate(df.gCode) if e == None]

In [31]:
len(geocode_errors)

4174

In [66]:
# errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)
# geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [39]:
# Yay! We've solved some errors!
len(geocode_errors)

3149

In [41]:
errors_df.iloc[geocode_errors].head(50)

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
12,653 LIBERTY AVE.,"CICCARELLI, MARC",308 LAGOON DR.,"COPIAQUE, NY 11726","308 LAGOON DR., COPIAQUE, NY 11726","653 LIBERTY ST. Jersey City, NJ",1,1,None
648,274-76 COLUMBIA AVE,"PERSAUD, PURAN & ANJAALI",274-276 COLUMBIA AVENUE,"JERSEY CITY,N.J. 07307","274-276 COLUMBIA AVENUE, JERSEY CITY,N.J. 07307","274-76 COLUMBIA AVE Jersey City, NJ",6,1,None
797,3633A JOHN F KENNEDY BLVD.,"SCALIA HOLDINGS, LLC",264 CENTRAL AVE.,"JERSEY CITY, NJ 07307","264 CENTRAL AVE., JERSEY CITY, NJ 07307","3633A JOHN F KENNEDY BLVD. Jersey City, NJ",5,1,None
1667,1074-1080 SUMMIT AVE,"1074 SUMMIT, LLC.",10 FULTON ST,"WEEHAWKEN, NJ 07086","10 FULTON ST, WEEHAWKEN, NJ 07086","1074-1080 SUMMIT AVE Jersey City, NJ",1,1,None
1821,19 PATERSON STREET -REAR,"BIRN, SUSAN",19 PATERSON STREET-REAR,"JERSEY CITY, N.J. 07307","19 PATERSON STREET-REAR, JERSEY CITY, N.J. 07307","19 PATERSON STREET -REAR Jersey City, NJ",1,1,None
2529,78A-80 THORNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A-80 THORNE ST. Jersey City, NJ",1,1,None
3427,430 ODGEN AVE.,"DEVIRGILLO, DAMIAN","430 OGDEN AVE., UNIT 3","JERSEY CITY, NJ 07307","430 OGDEN AVE., UNIT 3, JERSEY CITY, NJ 07307","430 ODGEN AVE. Jersey City, NJ",1,1,None
3511,608 TONNELE AVE.,"ADVANCE TRANSIT, LLC.",608 TONNELE AVE.,"JERSEY CITY, NJ 07307","608 TONNELE AVE., JERSEY CITY, NJ 07307","608 TONNELE AVE. Jersey City, NJ",1,1,None
3512,636 TONNELE AVE.,"OFRZ & OZAN SHELL, INC.",137-68 70TH AVE,"FLUSHING , N.Y. 11367","137-68 70TH AVE, FLUSHING , N.Y. 11367","636 TONNELE AVE. Jersey City, NJ",1,1,None
4240,978-80 SUMMIT AVE.,978-980 SUMMIT AVE. KBB LLC.,978-980 SUMMIT AVE.,"JERSEY CITY, NJ 07302","978-980 SUMMIT AVE., JERSEY CITY, NJ 07302","978-80 SUMMIT AVE. Jersey City, NJ",1,1,None


In [59]:
# There is no 'Tonnele Ave' – it's 'Tonnelle'
geolocator.geocode('LIENAU PL., Jersey City, NJ')

In [60]:
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TONNELE', 'TONNELLE')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC ADOO', 'MCADOO')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DRIVE', 'MARTIN LUTHER KING DRIVE')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COLUMBUS', 'CHRISTOPHER COLUMBUS')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIRST', '1st')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SECOND', '2nd')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THIRD', '3rd')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOURTH', '4th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTH', '5th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTH', '6th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVENTH', '7th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EIGHTH', '8th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINETH', '9th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TENTH', '10th')

In [14]:
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINTH', '9th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEENTH', '15th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTEENTH', '16th')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC DOUGALL', 'MCDOUGALL')

In [23]:
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTIER', 'CARPENTER')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOX HOUND', 'FOXHOUND')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIEDLER', 'SEIDLER')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIENAU', 'LINEAU')

In [26]:
errors_df.propertyFullAddress = errors_df.propertyLocation + ', Jersey City, NJ'

In [28]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [21]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [31]:
errors_df.iloc[geocode_errors].head(50)

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
648,274-76 COLUMBIA AVE,"PERSAUD, PURAN & ANJAALI",274-276 COLUMBIA AVENUE,"JERSEY CITY,N.J. 07307","274-276 COLUMBIA AVENUE, JERSEY CITY,N.J. 07307","274-76 COLUMBIA AVE, Jersey City, NJ",6,1,None
797,3633A JOHN F KENNEDY BLVD.,"SCALIA HOLDINGS, LLC",264 CENTRAL AVE.,"JERSEY CITY, NJ 07307","264 CENTRAL AVE., JERSEY CITY, NJ 07307","3633A JOHN F KENNEDY BLVD., Jersey City, NJ",5,1,None
1667,1074-1080 SUMMIT AVE,"1074 SUMMIT, LLC.",10 FULTON ST,"WEEHAWKEN, NJ 07086","10 FULTON ST, WEEHAWKEN, NJ 07086","1074-1080 SUMMIT AVE, Jersey City, NJ",1,1,None
1821,19 PATERSON STREET -REAR,"BIRN, SUSAN",19 PATERSON STREET-REAR,"JERSEY CITY, N.J. 07307","19 PATERSON STREET-REAR, JERSEY CITY, N.J. 07307","19 PATERSON STREET -REAR, Jersey City, NJ",1,1,None
2529,78A-80 THORNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A-80 THORNE ST., Jersey City, NJ",1,1,None
3427,430 ODGEN AVE.,"DEVIRGILLO, DAMIAN","430 OGDEN AVE., UNIT 3","JERSEY CITY, NJ 07307","430 OGDEN AVE., UNIT 3, JERSEY CITY, NJ 07307","430 ODGEN AVE., Jersey City, NJ",1,1,None
4240,978-80 SUMMIT AVE.,978-980 SUMMIT AVE. KBB LLC.,978-980 SUMMIT AVE.,"JERSEY CITY, NJ 07302","978-980 SUMMIT AVE., JERSEY CITY, NJ 07302","978-80 SUMMIT AVE., Jersey City, NJ",1,1,None
5409,223-221 HUTTON ST.,"ZANZIBARWALA, PRITESH & JIGNESH",221-223 HUTTON ST,"JERSEY CITY, N J 07307","221-223 HUTTON ST, JERSEY CITY, N J 07307","223-221 HUTTON ST., Jersey City, NJ",1,1,None
5561,882-882A SUMMIT AVE.,"BO BO ENTERPRISES 1, LLC",80 SHERMAN PLACE,"JERSEY CITY, NJ 07307","80 SHERMAN PLACE, JERSEY CITY, NJ 07307","882-882A SUMMIT AVE., Jersey City, NJ",1,1,None
5620,102 MANHATTAN AVE. (REAR),"M & A MANHATTAN AVE 102, LLC.",278 CENTRAL AVE.,"JERSEY CITY, NJ 07307","278 CENTRAL AVE., JERSEY CITY, NJ 07307","102 MANHATTAN AVE. (REAR), Jersey City, NJ",1,1,None


In [117]:
#errors_df.to_pickle('errors_df.pkl')

In [36]:
df_geocode_errors = [i for i,e in enumerate(df.gCode) if e == None]

In [38]:
df.iloc[df_geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
12,653 LIBERTY AVE.,"CICCARELLI, MARC",308 LAGOON DR.,"COPIAQUE, NY 11726","308 LAGOON DR., COPIAQUE, NY 11726","653 LIBERTY ST. Jersey City, NJ",1,1,None
112,3709 JOHN F KENNEDY BLVD.,"CARLTON URF, LLC","1000 PLAZA 2, 10 FL","JERSEY CITY, NJ 07311","1000 PLAZA 2, 10 FL, JERSEY CITY, NJ 07311","3709 KENNEDY BLVD. Jersey City, NJ",7,1,None
113,3707 JOHN F KENNEDY BLVD.,K & R REALTY 183 LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3707 KENNEDY BLVD. Jersey City, NJ",1,1,None
114,3705 JOHN F KENNEDY BLVD.,K R HEIGHTS LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3705 KENNEDY BLVD. Jersey City, NJ",2,1,None
115,3703 JOHN F KENNEDY BLVD.,K R HEIGHTS LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3703 KENNEDY BLVD. Jersey City, NJ",2,1,None
...,...,...,...,...,...,...,...,...,...
40786,162 OLD BERGEN,"162 OLD BERGEN AVE., LLC.",96 ARLINGTON AVE.,"JERSEY CITY, NJ 07305","96 ARLINGTON AVE., JERSEY CITY, NJ 07305","162 OLD BERGEN Jersey City, NJ",1,1,None
41496,1524 JOHN F KENNEDY BLVD.,"FERNANDES, SHONNON",1524 KENNEDY BLVD.,"JERSEY CITY, N.J. 07305","1524 KENNEDY BLVD., JERSEY CITY, N.J. 07305","1524 KENNEDY BLVD. Jersey City, NJ",1,1,None
41497,1526 JOHN F KENNEDY BLVD.,"SOLIMAN, AIDA NAGEEB",1526 KENNEDY BLVD.,"JERSEY CITY, N.J. 07306","1526 KENNEDY BLVD., JERSEY CITY, N.J. 07306","1526 KENNEDY BLVD. Jersey City, NJ",1,1,None
41498,1528 JOHN F KENNEDY BLVD.,"SALIB, HANY",1528 KENNEDY BLVD.,"JERSEY CITY, NJ 07305","1528 KENNEDY BLVD., JERSEY CITY, NJ 07305","1528 KENNEDY BLVD. Jersey City, NJ",1,1,None


In [5]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
12,653 LIBERTY AVE.,"CICCARELLI, MARC",308 LAGOON DR.,"COPIAQUE, NY 11726","308 LAGOON DR., COPIAQUE, NY 11726","653 LIBERTY AVE., Jersey City, NJ",1,1,"(653, Liberty Avenue, Jersey City, Hudson Coun..."
112,3709 JOHN F KENNEDY BLVD.,"CARLTON URF, LLC","1000 PLAZA 2, 10 FL","JERSEY CITY, NJ 07311","1000 PLAZA 2, 10 FL, JERSEY CITY, NJ 07311","3709 JOHN F KENNEDY BLVD., Jersey City, NJ",7,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
113,3707 JOHN F KENNEDY BLVD.,K & R REALTY 183 LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3707 JOHN F KENNEDY BLVD., Jersey City, NJ",1,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
114,3705 JOHN F KENNEDY BLVD.,K R HEIGHTS LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3705 JOHN F KENNEDY BLVD., Jersey City, NJ",2,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
115,3703 JOHN F KENNEDY BLVD.,K R HEIGHTS LLC,P.O. BOX 4443,"WARREN, NJ 07059","P.O. BOX 4443, WARREN, NJ 07059","3703 JOHN F KENNEDY BLVD., Jersey City, NJ",2,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
...,...,...,...,...,...,...,...,...,...
40786,162 OLD BERGEN,"162 OLD BERGEN AVE., LLC.",96 ARLINGTON AVE.,"JERSEY CITY, NJ 07305","96 ARLINGTON AVE., JERSEY CITY, NJ 07305","162 OLD BERGEN, Jersey City, NJ",1,1,None
41496,1524 JOHN F KENNEDY BLVD.,"FERNANDES, SHONNON",1524 KENNEDY BLVD.,"JERSEY CITY, N.J. 07305","1524 KENNEDY BLVD., JERSEY CITY, N.J. 07305","1524 JOHN F KENNEDY BLVD., Jersey City, NJ",1,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
41497,1526 JOHN F KENNEDY BLVD.,"SOLIMAN, AIDA NAGEEB",1526 KENNEDY BLVD.,"JERSEY CITY, N.J. 07306","1526 KENNEDY BLVD., JERSEY CITY, N.J. 07306","1526 JOHN F KENNEDY BLVD., Jersey City, NJ",1,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."
41498,1528 JOHN F KENNEDY BLVD.,"SALIB, HANY",1528 KENNEDY BLVD.,"JERSEY CITY, NJ 07305","1528 KENNEDY BLVD., JERSEY CITY, NJ 07305","1528 JOHN F KENNEDY BLVD., Jersey City, NJ",1,1,"(John F. Kennedy Boulevard, Greenville, Jersey..."


In [40]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
648,274-76 COLUMBIA AVE,"PERSAUD, PURAN & ANJAALI",274-276 COLUMBIA AVENUE,"JERSEY CITY,N.J. 07307","274-276 COLUMBIA AVENUE, JERSEY CITY,N.J. 07307","274-76 COLUMBIA AVE, Jersey City, NJ",6,1,None
797,3633A JOHN F KENNEDY BLVD.,"SCALIA HOLDINGS, LLC",264 CENTRAL AVE.,"JERSEY CITY, NJ 07307","264 CENTRAL AVE., JERSEY CITY, NJ 07307","3633A JOHN F KENNEDY BLVD., Jersey City, NJ",5,1,None
1667,1074-1080 SUMMIT AVE,"1074 SUMMIT, LLC.",10 FULTON ST,"WEEHAWKEN, NJ 07086","10 FULTON ST, WEEHAWKEN, NJ 07086","1074-1080 SUMMIT AVE, Jersey City, NJ",1,1,None
1821,19 PATERSON STREET -REAR,"BIRN, SUSAN",19 PATERSON STREET-REAR,"JERSEY CITY, N.J. 07307","19 PATERSON STREET-REAR, JERSEY CITY, N.J. 07307","19 PATERSON STREET -REAR, Jersey City, NJ",1,1,None
2529,78A-80 THORNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A-80 THORNE ST., Jersey City, NJ",1,1,None
...,...,...,...,...,...,...,...,...,...
38815,4 CONSTELLATION PL.,"PRAKASH, NIKHIL","4 CONSTELLATION PL.,#404","JERSEY CITY, NJ 07305","4 CONSTELLATION PL.,#404, JERSEY CITY, NJ 07305","4 CONSTELLATION PL., Jersey City, NJ",1,29,None
38817,4 CONSTELLATION PL.,"DOBSON, MABLE E & TANYA",2 BUCKS LN.,"MARLBORO, NJ 07746","2 BUCKS LN., MARLBORO, NJ 07746","4 CONSTELLATION PL., Jersey City, NJ",1,29,None
38820,4 CONSTELLATION PL.,"BANDYOPADHYAY, RAHUL",806 MORRIS TPKE APT 1A,"SHORT HILLS, NJ 07078","806 MORRIS TPKE APT 1A, SHORT HILLS, NJ 07078","4 CONSTELLATION PL., Jersey City, NJ",1,29,None
40216,12 DANFORTH AVE. THO,"RIVERA, CARMEN & MAGDALENA",12 DANFORTH AVE.,"JERSEY CITY, NJ 07305","12 DANFORTH AVE., JERSEY CITY, NJ 07305","12 DANFORTH AVE. THO, Jersey City, NJ",1,1,None


In [19]:
# reassigning the dataframe to only geocoded addresses
df = df.append(errors_df.dropna(axis=0, subset=['gCode']))

In [22]:
# reassigning the errors_df to the remaining errors
errors_df = errors_df.iloc[geocode_errors]

In [29]:
df['latitude'] = [g.latitude for g in df.gCode]
df['longitude'] = [g.longitude for g in df.gCode]

In [31]:
df.to_pickle('JC_residential.pkl')
errors_df('errors_df.pkl')

In [34]:
df.to_csv('JC_residential.csv', index=None)